<a href="https://colab.research.google.com/github/gjrkdgh0772/Finance/blob/main/lec05_%EA%B8%88%EC%9C%B5_pandas_mission.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size=6><b>Lec02 : 금융을 위한 Pandas 활용 </b></font>

### 주식 용어
<pre>
* <b>시가총액</b> : 시가총액은 기업의 가치를 결정하는 지표
          시가총액 = 주식 수 × 주식 1주당 가격

* <B>KOSPI</B> : 한국증권거래소에 상장되어 거래되는 모든 주식의 시가총액을 합쳐서 주식시장 전체 장세의 흐름을 나타내는 지수
          KOSPI 지수 = (현재 시가총액 합 / 1980. 1. 4 기준 시가총액 합) × 100
 
* <B> KOSDAQ</B> : 중소기업이나 신생 벤처기업을 위한 1996년 첨단 기술주 중심인 미국의 나스닥(NASDAQ) 시장을 본떠 만든 대한민국의 주식시장

* <B> KOSPI 200</B> : 어업, 광업, 제조업, 전기가스업, 건설업, 서비스업, 통신업 등 8개의 산업군에서 각 산업을 대표할 수 있는 200개의 종목을 뽑아서 만든 지수

* <B> KODEX 200</B> : 삼성자산운용에서 운용하는 ETF로, KOSPI 200을 추종(복제)하는 ETF

* <B> ETF</B> : 특정 주가지수 또는 금, 채권, 원유와 같은 특정 자산가격를 그대로 추종하는 성격을 지닌 금융상품
 </pre>

In [ ]:
from pykrx import stock
from pykrx import bond
import FinanceDataReader as fdr

from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings(action='ignore')

#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
#plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
sns.set()

#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)



# Data Load by pykrx

In [ ]:
tickers = stock.get_market_ticker_list("20220225", market="KOSDAQ") #KOSPI, KOSDAQ, KONEX
print(tickers[:5])

['060310', '054620', '265520', '211270', '032790']


In [ ]:
종목명 = stock.get_market_ticker_name('035760')
print(종목명)

CJ ENM


In [ ]:
df = stock.get_market_ohlcv("2021-01-01", "2021-12-31", "005930")
df.head(3)

,시가,고가,저가,종가,거래량
날짜,,,,,
2021-01-04,81000,84400,80200,83000,38655276
2021-01-05,81600,83900,81600,83900,35335669
2021-01-06,83300,84500,82100,82200,42089013


* 컬럼 이름 변경

In [ ]:
df.columns = ['open','high','low','close','volumn']

In [ ]:
df.isna().sum()

open      0
high      0
low       0
close     0
volumn    0
dtype: int64

# 금융을 위한 pandas

## shift(N)
* N일가격 컬럼에 가져오기
* <font size=4 color=red><b>전일 대비 가격 증감에 주로 사용 

In [ ]:
df['close_shift'] = df['close'].shift(1)
df.head()

,open,high,low,close,volumn,close_shift
날짜,,,,,,
2021-01-04,81000,84400,80200,83000,38655276,NaN
2021-01-05,81600,83900,81600,83900,35335669,83000.0
2021-01-06,83300,84500,82100,82200,42089013,83900.0
2021-01-07,82800,84200,82700,82900,32644642,82200.0
2021-01-08,83300,90000,83000,88800,59013307,82900.0


In [ ]:
df['close'].shift(-1)

날짜
2021-01-04    83900.0
2021-01-05    82200.0
2021-01-06    82900.0
2021-01-07    88800.0
2021-01-08    91000.0
               ...   
2021-12-24    80200.0
2021-12-27    80300.0
2021-12-28    78800.0
2021-12-29    78300.0
2021-12-30        NaN
Name: close, Length: 248, dtype: float64

## df["Close"] - df["Close"].shift(1)

In [ ]:
last_day = df["Close"].shift(1)
today    = df["Close"]
df["pct_chg1"] =   ( (today - last_day) / last_day ) * 100
df.head(3)

KeyError: 'Close'

## diff(N)
* 변화량 : 현재가격 - N일가격 
* <font size=4 color=red><b> N일 대비 가격 증감에 주로 사용
* <font size=4 color=blue><b> df['close'].diff(1) = </font>  <font size=4 color=red><b> df['close'] - df['colse'].shift(1)

In [ ]:
df['close_diff'] = df['close'].diff(1)
df['close-close_shift'] = df['close'] - df['close_shift']
df.head()

## pct_change(N)
* 백분위 변화율 : 현재가격 - N일가격 
* <font size=4 color=red><b>전일 대비 수익률에 주로 사용

In [ ]:
df['pct_chg'] = df['close'].pct_change()
df.head()

## rolling(window=N)
*  N일 간격으로 그룹화해 가격을 집계
* <font size=4 color=red><b> 이동평균선에 주로 사용

In [ ]:
df['close'].rolling(window=5).mean()

In [ ]:
df['close_ma5']   = df['close'].rolling(window=5).mean()
df['close_ma20']  = df['close'].rolling(window=20).mean()
df['close_ma60']  = df['close'].rolling(window=60).mean()
df['close_ma120'] = df['close'].rolling(window=120).mean()
df.head(3)

## resample()
* 업샘플링 : 보간법 사용해 샘플수 늘리기
* 다운샘플링 : 집계 사용해 샘플수 줄이기

In [ ]:
print(df.shape)
df['close'].resample(rule='M').mean()  #월평균
df.head(3)

In [ ]:
print(df.shape)
print( df['close'].resample(rule='MS').first() )  #매월 초 가격
print( df['close'].resample(rule='M').last() )    #매월 말 가격

# [실습] FinanceDataReader

* 시가총액 : 시가총액은 기업의 가치를 결정하는 지표
          시가총액 = 주식 수 × 주식 1주당 가격

* KOSPI : 한국증권거래소에 상장되어 거래되는 모든 주식의 시가총액을 합쳐서 주식시장 전체 장세의 흐름을 나타내는 지수
          KOSPI 지수 = (현재 시가총액 합 / 1980. 1. 4 기준 시가총액 합) × 100
 
*  KOSDAQ : 중소기업이나 신생 벤처기업을 위한 1996년 첨단 기술주 중심인 미국의 나스닥(NASDAQ) 시장을 본떠 만든 대한민국의 주식시장

*  KOSPI 200 : 어업, 광업, 제조업, 전기가스업, 건설업, 서비스업, 통신업 등 8개의 산업군에서 각 산업을 대표할 수 있는 200개의 종목을 뽑아서 만든 지수

*  KODEX 200 : 삼성자산운용에서 운용하는 ETF로, KOSPI 200을 추종(복제)하는 ETF

*  ETF : 특정 주가지수 또는 금, 채권, 원유와 같은 특정 자산가격를 그대로 추종하는 성격을 지닌 금융상품

## Data Load

### FinanceDataReader 사용 시 

In [ ]:
import FinanceDataReader as fdr
df_ss = fdr.DataReader('005930', '2021-01-01', '2021-12-31', exchange='KRX') 
df_ss.head(3)

* 현재 investing 에러..

In [ ]:
# ! pip install -U finance-datareader 

In [ ]:
# df_aapl = fdr.DataReader('AMZN', '2021-01-01', '2021-12-31')
# df_aapl = fdr.DataReader('AAPL', '2021-01-01', '2021-12-31')

In [ ]:
Image(filename='./img/lec05_2.png', width = 600)

In [ ]:
Image(filename='./img/lec05_1.png', width = 600)

### pandas_datareader 사용 시

In [ ]:
from pandas_datareader import data as pdr

* 상장 회사 정보 가져오기

In [ ]:
ticker_df = fdr.StockListing('KRX')  # 'KRX', 'KOSPI', 'KODAQ', 'KONEX'
ticker_df.head()

In [ ]:
ticker_df = fdr.StockListing('NASDAQ')  # 'NASDAQ', 'NYSE', 'AMEX' and 'S&P500', 'SSE'(상해), 'SZSE'(심천), 'HKEX'(홍콩), 'TSE'(도쿄)
ticker_df.head()

In [ ]:
# df_ss = pdr.DataReader("005930.KS" ,start='2018-01-01' ,end='2019-12-31' ,data_source='yahoo' )
df_ss = pdr.get_data_yahoo('005930.KS', '2021-01-01', '2021-12-31')
print(df_ss.shape)
df_ss.head(3)

In [ ]:
df_ts = pdr.get_data_yahoo('TSLA', '2021-01-01', '2021-12-31')
print(df_ts.shape)
df_ts.head(3)

### yfinance (pandas_datareader) 사용 시

In [ ]:
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()

In [ ]:
df_ts = yf.download("TSLA", start="2021-01-01", end="2021-12-31")
df_ts.head()

In [ ]:
plt.figure(figsize=(9, 6))
plt.title('Samsung vs TSLA close')
plt.plot(df_ss.index, df_ss['Close'], label='samsung')
plt.plot(df_ts.index, df_ts['Close'], label='tsla')
plt.legend()
plt.show()

## 일간변동률(Daily Percent Change) 
* <font color=red size=4><b>가격이 다른 두 주가의 수익률 비교
* <font color=red>오늘 - 기준일 / 기준일 * 100
* ref : https://www.investopedia.com/terms/p/percentage-change.asp

<pre>
* increase   = New Number - Original Number
* increase%  = Increase ÷ Original Number × 100.
* decrease   = Original Number - New Number
* decrease%  = Decrease ÷ Original Number × 100

  

### df["Close"].pct_change() 

In [ ]:
df_ss["pct_chg2"] = df_ss["Close"].pct_change() 
df_ts["pct_chg2"] = df_ts["Close"].pct_change() 
df_ss.head(3)

### 결측 채우기

In [ ]:
df_ss.fillna(method='bfill', inplace=True)
df_ts.fillna(method='bfill', inplace=True)
df_ss.head()

### 차트

In [ ]:
plt.figure(figsize=(9, 6))
# plt.subplot(2, 1, 1)
plt.title('Samsung vs MS close')
plt.plot(df_ss.index, df_ss['pct_chg2'], label='samsung')
plt.plot(df_ts.index, df_ts['pct_chg2'], label='TSLA')
plt.legend()

## 일간 변동률 누적합

In [ ]:
plt.figure(figsize=(9, 6))
plt.title('Samsung Elec')
plt.plot(df_ss.index  , df_ss['change'].cumsum(), label='samsung')
plt.plot(df_ts.index  , df_ts['change'].cumsum(), label='tsla')
plt.legend()

# [실습] 최대낙폭, 최대손실폭(Maximum DrowDown) : MDD 
* 특정 기간에서 발생한 최고점에서 최저점까지의 가장 큰 손실 == 최대 얼마의 손실이 날 수 있는가?
* (65-130)/130=-50%

<table>
<tr>
<td><img src='https://downloads.intercomcdn.com/i/o/79244225/7dad4f4f919e2be8a4a791e6/MDD.png?expires=1619851026&signature=aafb18bbd3ed5f6946ad53a5fb9f19d6dbc7316703c94667bc422babd38a2200'></td><br>
<td><img src='https://downloads.intercomcdn.com/i/o/79243103/d9a69bea18818ef2f126a6f8/%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA+2018-10-04+15.48.43.png?expires=1619851026&signature=dcce134c2a4882b00790a53d5d8cba27350f8f48f8c59d681be007e99def032c'></td>
</tr>
</table>

## Data Load 

In [ ]:
import FinanceDataReader as fdr
ohlcv_df   = fdr.DataReader('005930', '2021')  # 삼성전자(005930) , 2017-01-01 ~ Now
ohlcv_df.columns = [ str.lower() for str in ohlcv_df.columns]
print(ohlcv_df.shape)
ohlcv_df.head()

## 1년 기준 최고가 

In [ ]:
ohlcv_df['peak'] = ohlcv_df['close'].rolling(window=252, min_periods=1).max()
ohlcv_df.head()

## 최고가 대비 종가 비율

In [ ]:
# 최고치 대비 현재 종가가 얼마나 하락했는가?
ohlcv_df['close_pick_rate'] =  (ohlcv_df['close'] - ohlcv_df['peak']) / ohlcv_df['peak']
ohlcv_df.head()

## MDD

In [ ]:
mdd = ohlcv_df.rolling(window=252, min_periods=1)['close_pick_rate'].min()
mdd.min()

In [ ]:
plt.figure(figsize=(9, 6))
plt.title('KOSPI')
plt.plot(ohlcv_df['close_pick_rate'], label='close_pick_rate')
plt.plot(mdd, label='mdd')
plt.legend()

### 공통함수 생성

* 차트 그리기

In [ ]:
def myplot(df, nrows=5,  kind='line', title='제목', labels=False):  
    if nrows > 0:
        print(df.head(nrows))
    else:
        print(df.tail(nrows))
    
    if labels:
        cols = df.columns
        for i, col in enumerate(cols):
            df[col].plot(label=labels[i], kind=kind)
    else :
        df.plot(kind=kind)
    
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
def myplotmix(df1, df2, y1='ylabel-1', y2='ylabel-2', kind1='bar', kind2='line', title='제목',  nrows1=5, nrows2=5,  labels=False):  
    ytl=['b', 'g', 'r', 'c', 'm', 'y', 'k', 'w']

    if nrows1 > 0:
        print(df1.head(nrows1))
    elif nrows1 < 0:
        print(df1.tail(nrows1))
    if nrows2 > 0:
        print(df2.head(nrows2))
    else:
        print(df2.tail(nrows2))
    fig, ax1 = plt.subplots(figsize=(12, 4))
    ax2=ax1.twinx()
    if kind1 == 'line':
        df1.plot(kind=kind1, ax=ax2, marker='d',color='#e35f62')  #color='y', 
        df2.plot(kind=kind2, ax=ax1)  #bar
    else:
        df1.plot(kind=kind1, ax=ax1,color='#bcbd22')
        df2.plot(kind=kind2, ax=ax2) #color='y',
        
    # df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    # ax1.set_xticklabels(df['date'])    
    ax1.yaxis.tick_right()
    ax2.yaxis.tick_left()
    ax1.set_ylabel(y1) #, color='red')
    ax2.set_ylabel(y2) #, color='red')
    ax1.legend(loc=1)
    ax2.legend(loc=2)
    plt.title(title)
    ax1.grid(False)
    ax2.grid(False)
    plt.show()

# 키움증권 수익률 계산 예시

* <b>투자원금평잔</b> = 평가 기간초 자산액계 + 기간내 총입출금고평잔
* <b>평가손익</b> = 기간말자산액 - 기간초자산액 + 출금 - 입금 + 총출고 - 총입고 + 매도 - 매수
* <b>수익률</b> = (평가손익/투자원금평잔) * 100

<img src='https://t1.daumcdn.net/cfile/tistory/99FF84455EB80BED05'>